# Module ulpp

De `ulpp` library verzorgt de codering en decodering van berichten in het binaire Cayenne LPP formaat.
Voor een beschrijving van dit formaat, zie:

* https://docs.mydevices.com/docs/lorawan/cayenne-lpp
* https://github.com/myDevicesIoT/CayenneLPP
* https://github.com/smlng/pycayennelpp
    * deze beschrijft meer sensor-formaten dan de oorspronkelijk LPP versie


De `ulpp` library is een onafhankelijke implementatie van de codering en decodering, bedoeld om het gebruik van LPP voor de microbit eenvoudig te houden.

De belangrijke elementen van deze library zijn:

* `LppFrame(prefix, maxsize)`
* `frame.add_digital_input(channel, value)` - etc.
    * met varianten voor elk type sensor
* `frame.to_bytes())`
* `bytes_to_dict(bytes)`
* `dict_to_bytes(dict)`

Via `LppFrame` en `frame.add_xxsensor(channel, value)` bouw je een array van getallen op die je als byte-array kunt versturen.

Dit byte-array kun je ook omzetten (decoderen) met `bytes_to_dict()` in een Python dictionary - die je vervolgens kunt omzetten in JSON, voor een MQTT bericht.
Via de codering `dict_to_bytes()` kun je een Python dictionary omzetten naar een LPP byte-array.

* `dict_to_bytes()` verwerkt in deze implementatie (voorlopig) alleen actuator-data: deze is bedoeld voor downlink-berichten.

Voorbeelden van het gebruik:

**Opbouwen van een reeks sensorwaarden**, en versturen als byte-array. (Uplink-berichten in een IoT-knoop.)

```Python
prefix = bytes([1,2,3]) # prefix before LPP data
frame = LppFrame(data=prefix, maxsize=60)
frame.add_digital_input(0, 1)
frame.add_digital_output(1, 1)
frame.add_analog_input(2, 1234)
frame.add_analog_output(4, -1234)
frame.add_luminosity(7, 345)
frame.add_presence(8, 1)
frame.add_barometer(9, 10230)
frame.add_temperature(5, int(23.4*10))
frame.add_digital_input(10, 1)
radio.send_bytes(frame.to_bytes())
```

**Omzetten van een ontvangen (binair) LPP-bericht in acties** - afhandelen van actuator-data.

(In een IoT-knoop)

**Omzetten (decoderen) van een ontvangen byte-array** in een JSON-string, om te versturen via MQTT. (Uplink-berichten in de gateway.)




**Omzetten (coderen) van een ontvangen JSON-string** in een binair LPP byte-array (downlink-berichten in de gateway).

```Python
lpp_dict = json.loads(msg)            # from JSON string to Python object
lpp_bytes = dict_to_bytes(lpp_dict)   # from Python object to LPP byte-array
header_bytes = bytes([0x0B, nodeID // 256, nodeID % 256, 0, 0]) # downlink header  
radio.send_bytes(header_bytes + lpp_bytes)    # send header followed by LPP data
```

In [29]:
class LppFrame(object):

    # some assumptions:
    # - the value-parameters are in the LPP-required format and range
    #   i.e. scaling is done by the caller (if needed)
    #   all value-parameters are int
    
    def __init__(self, data=b'', maxsize=32):
        self.buffer = bytearray(data)
        self.maxsize = maxsize
        self.pos = 0

    def __str__(self):
        return str(list(self.buffer))
    
    def to_bytes(self):
        return self.buffer
    
    def add_byte(self, channel, tag, value):
        if len(self.buffer) + 3 > self.maxsize:
            raise OverflowError
        self.buffer.append(channel)
        self.buffer.append(tag)
        self.buffer.append(value)

    def add_unsigned_int16(self, channel, tag, value):
        if len(self.buffer) + 4 > self.maxsize:
            raise OverflowError        
        self.buffer.append(channel)
        self.buffer.append(tag)
        if value >= 65536:
            value = value % 65536
        (hi, lo) = divmod(value, 256)
        self.buffer.append(hi)
        self.buffer.append(lo)

    def add_signed_int16(self, channel, tag, value):
        while value < 0:
            value = value + 65536
        self.add_unsigned_int16(channel, tag, value)

    def add_digital_input(self, channel, value):
        self.add_byte(channel, 0, value)

    def add_digital_output(self, channel, value):
        self.add_byte(channel, 1, value)
        
    def add_analog_input(self, channel, value):
        self.add_signed_int16(channel, 2, value)

    def add_analog_output(self, channel, value):
        self.add_signed_int16(channel, 3, value)

    def add_luminosity(self, channel, value):
        self.add_unsigned_int16(channel, 101, value)

    def add_presence(self, channel, value):
        self.add_byte(channel, 102, value)

    def add_temperature(self, channel, value):
        # temperature: 0.1C, signed int
        self.add_signed_int16(channel, 103, value)

    def add_humidity(self, channel, value):
        # rel. humidity: 0.5% unsigned byte
        self.add_byte(channel, 104, value)

    def add_barometer(self, channel, value):
        # barometric pressue: 0.1 hPa unsigned int16
        self.add_unsigned_int16(channel, 115, value)

def bytes_to_dict(data):
    
    buffer = data
    pos = 0

    def nextbyte():
        nonlocal pos
        if pos >= len(buffer):
            raise OverflowError
        value = buffer[pos]
        pos = pos + 1
        return value
    
    def nextunsignedint():
        hi = nextbyte()
        lo = nextbyte()
        value = hi * 256 + lo
        return value 
    
    def nextint():
        value = nextunsignedint()
        if value > 32767:
            value = value - 65536
        return value
    

    pos = 0
    obj = {}

    while pos < len(buffer):
        channel = nextbyte()
        tag = nextbyte()
        if tag == 0:
            value = nextbyte()
            obj[channel] = {'dIn': value}
        elif tag == 1:
            value = nextbyte()
            obj[channel] = {'dOut': value}                
        elif tag == 2:
            value = nextint()
            obj[channel] = {'aIn': value}                
        elif tag == 3:
            value = nextint()
            obj[channel] = {'aOut': value}
        elif tag == 101:
            value = nextunsignedint()
            obj[channel] = {'luminosity': value}                
        elif tag == 102:
            value = nextbyte()
            obj[channel] = {'presence': value}                
        elif tag == 103:
            value = nextint()                
            obj[channel] = {'temperature': value}                
        elif tag == 104:
            value = nextbyte()
            obj[channel] = {'humidity': value}                
        elif tag == 115:
            value = nextunsignedint()
            obj[channel] = {'barometer': value}
            
    return obj
 
def dict_to_bytes (obj):
    lpp = LppFrame()

    for channel in obj:
        item = obj[channel]  # item is an object with a single key...
        if type(channel) is str:
            channel = int(channel)
        for key in item:
            if key == 'dOut':
                lpp.add_digital_output(channel,  item[key])
            elif key == 'aOut':
                lpp.add_analog_output(channel,  item[key])
            else:
                # not implemented, raise exception? only output allowed
                raise ValueError('only output values allowed in actuator msg')

    return lpp.to_bytes()

* er is geen controle op de waarden die aangeboden worden
* er is geen controle op de tag-waarden?
    * als het niet klopt: afbreken.... exception (melding van tag)
    * er kan ook een byte te weinig zijn.... exception
* 

De code hieronder is de actuele code van ulpp.py (zie ook TW)

Bij de decodering zetten we een LPP-bytestring om in een Python dictionary waarde. Deze gebruiken we (i) voor het interpreteren van de ontvangen actuator-waarden, in de IoT-knoop; en (ii), voor het omzetten naar JSON, in de gateway.

Beide classes kunnen eigenlijk in 1 bestand geplaatst worden, omdat we deze combinatie vaak nodig hebben. En het is hiermee ook compleet.

(... niet helemaal: we moeten in de gateway ook een **JSON-waarde omzetten in een LPP-bytestring**...)

## Testen van de code

De onderstaande testen kunnen uitgevoerd worden in Jupyter Notebook.

### Opbouwen van een binaire LPP-waarde

Deze code vind je in de IoT-knoop, bij het versturen van de sensorwaarden.
En mogelijk bij het opbouwen van een LPP-waarde met actuator-waarden, vanuit een toepassing.

In [30]:
prefix = bytes([1,2,3])
frame = LppFrame(data=prefix, maxsize=60)
frame.add_digital_input(0, 1)
frame.add_digital_output(1, 1)
frame.add_analog_input(2, 1234)
frame.add_analog_output(4, -1234)
frame.add_luminosity(7, 345)
frame.add_presence(8, 1)
frame.add_barometer(9, 10230)
frame.add_temperature(5, int(23.4*10))
frame.add_digital_input(10, 1)
  # radio.send_bytes(frame.to_bytes())
buffer = frame.to_bytes()
print(list(buffer))

[1, 2, 3, 0, 0, 1, 1, 1, 1, 2, 2, 4, 210, 4, 3, 251, 46, 7, 101, 1, 89, 8, 102, 1, 9, 115, 39, 246, 5, 103, 0, 234, 10, 0, 1]


### van LPP-bytes naar JSON

In de gateway worden ontvangen (binaire) sensor-berichten omgezet in JSON-formaat, om te versturen via MQTT.

In [34]:
import json

In [36]:
lppdata = bytes_to_dict(buffer[3:]) # skip prefix
lppdata

{0: {'dIn': 1},
 1: {'dOut': 1},
 2: {'aIn': 1234},
 4: {'aOut': -1234},
 7: {'luminosity': 345},
 8: {'presence': 1},
 9: {'barometer': 10230},
 5: {'temperature': 234},
 10: {'dIn': 1}}

In [57]:
jsondata = json.dumps(lppdata)
jsondata

'{"0": {"dIn": 1}, "1": {"dOut": 1}, "2": {"aIn": 1234}, "4": {"aOut": -1234}, "7": {"luminosity": 345}, "8": {"presence": 1}, "9": {"barometer": 10230}, "5": {"temperature": 234}, "10": {"dIn": 1}}'

### Van JSON naar LPP-bytes

In [59]:
lppdata1 = json.loads(jsondata)
lppdata1

{'0': {'dIn': 1},
 '1': {'dOut': 1},
 '2': {'aIn': 1234},
 '4': {'aOut': -1234},
 '7': {'luminosity': 345},
 '8': {'presence': 1},
 '9': {'barometer': 10230},
 '5': {'temperature': 234},
 '10': {'dIn': 1}}

In [62]:
lppbytes1 = dict_to_bytes(lppdata1)
lppbytes1

ValueError: only output values allowed in actuator msg

Bovenstaande werkt niet in de (beperkte) implementatie van dict_to_bytes. Dit werkt wel als we de dictionary beperken tot alleen actuator-waarden.

In [66]:
lppbytes1 = dict_to_bytes({'1': {'dOut': 1}, '4': {'aOut': -1234}})
list(lppbytes1)

[1, 1, 1, 4, 3, 251, 46]

Nog een extra voorbeeld, met alleen actuator-waarden.

In [71]:
lpp = LppFrame()
lpp.add_analog_output(3, 123)
lpp.add_digital_output(5, 1)
lpp.add_analog_output(6, -12)

lppbuf = lpp.to_bytes()

lppdict = bytes_to_dict(lppbuf)

In [72]:
list(dict_to_bytes(lppdict))

[3, 3, 0, 123, 5, 1, 1, 6, 3, 255, 244]

**Opmerking.** Het is ook mogelijk om in de microbit de codering en codering uit te schrijven, als we niet genoeg geheugen hebben voor zo'n library. Zo complex is dat nu ook weer niet. Voor de microbit V2 is het geheugen niet zo'n probleem, maar voor de V1 (in Python) wel.

(In de RF69-versie is die codering en codering ook uitgeschreven, zonder gebruik van een library.)